In [2]:
# !pip install olefile
# !pip install docx
# !pip install exceptions
# !pip install python-pptx
# !pip install python-docx
# !pip install tika

In [5]:
!pip install pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 37.6 MB/s eta 0:00:0000:0100:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [2]:
#hwp, pptx, docx, xls, csv, xlsx. pdf -> txt로 변환

In [6]:
import olefile
import pandas as pd
import glob
from os import rename, listdir
import os
from datetime import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [4]:
#리스트 -> 문자열
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [125]:
# 정규표현식 한글 남겨두고 특수문자, \r\n 제거하여 관련 정규문
import re

def remove_special_characters(text):
    return re.sub(r'[^0-9a-zA-Z가-힣\s\r\n]', '', text).strip()

In [5]:
target_path = input("검사하실 target문서의 directory경로를 입력하시요: ")
compare_path = input("문서 뭉치 directory경로를 입력하시오: ")

검사하실 target문서의 directory경로를 입력하시요: /content/drive/MyDrive/final project/target
문서 뭉치 directory경로를 입력하시오: /content/drive/MyDrive/final project/compare


In [127]:
#검사하실 target문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{target_path}/'+'*.hwp')


hwp_df = pd.DataFrame() 
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{target_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
  prs = Presentation(f"{i}")

  

  data = []
  for slide in prs.slides:
    
    for shape in slide.shapes:
        if not shape.has_text_frame:
              continue
        text_frame = shape.text_frame
        for paragraph in text_frame.paragraphs:
              for run in paragraph.runs:

                  data.append(run.text)


  pptx_df['title'] = np.nan
  pptx_df['content'] = np.nan
  pptx_df['extension'] = np.nan
  pptx_df["content"] = [listToString(data)]
  pptx_df["title"] = names          
  pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{target_path}/'+'*.docx')
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'


# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{target_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
  pdf_path = f"{i}"
  parsed = parser.from_file(pdf_path)
  txt = open('output.txt', 'w', encoding = 'utf-8')
  # output.txt에 pdf파일 내용을 write
  content_list.append(parsed['content'])

  txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# xlsx 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{target_path}/'+'*.xlsx')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_xlsx_df = pd.DataFrame()


for i in filelist_xlsx:
    csv_xlsx_df = pd.read_excel(f"{i}")
  
full_text = []
for column in csv_xlsx_df:
    for item in csv_xlsx_df[column]:

        full_text.append(str(item))
csv_xlsx_df = csv_xlsx_df.head(1)

csv_xlsx_df['title'] = np.nan
csv_xlsx_df['title'] = listToString(names)
csv_xlsx_df['content'] = listToString(full_text)
csv_xlsx_df['extension'] = 'csv'
csv_xlsx_df = csv_xlsx_df[['title','content','extension']]

In [71]:
df_target_merged = pd.concat([hwp_df, pptx_df, docx_df,pdf_df,csv_xlsx_df])
df_target_merged = df_target_merged.reset_index()
df_target_merged = df_target_merged.drop(['index'],axis=1)

In [147]:
for i in range(len(df_target_merged['content'])):
  df_target_merged['content'].loc[i] = remove_special_characters(df_target_merged['content'].loc[i])

In [148]:
df_target_merged

,title,content,extension
0,(공지)기업데이터를 활용한 AI 취업 부트ᄏ...,No 일자 요일 교과목 단원명 강사 세부 교육내용 교육시간 장비 교재 구분\n\n ...,pdf


In [142]:
#비교 대상 문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{compare_path}/'+'*.hwp')


hwp_df = pd.DataFrame() 
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{compare_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
  prs = Presentation(f"{i}")

  

  data = []
  for slide in prs.slides:
    
    for shape in slide.shapes:
        if not shape.has_text_frame:
              continue
        text_frame = shape.text_frame
        for paragraph in text_frame.paragraphs:
              for run in paragraph.runs:

                  data.append(run.text)


  pptx_df['title'] = np.nan
  pptx_df['content'] = np.nan
  pptx_df['extension'] = np.nan
  pptx_df["content"] = [listToString(data)]
  pptx_df["title"] = names          
  pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{compare_path}/'+'*.docx')
# target = 'target.docx'
# base = 0
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
# index = df[df['title'] == target].index[0]
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'

# xls 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xls')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_df = pd.DataFrame()

full_text = []

for i in filelist_xlsx:
    csv_df = pd.read_csv(f"{i}")
xw
for column in csv_df:
    for item in csv_df[column]:

        full_text.append(str(item))



csv_df['title'] = np.nan
csv_df['title'] = listToString(names)
csv_df['content'] = listToString(full_text)
csv_df['extension'] = 'csv'
csv_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{compare_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
  pdf_path = f"{i}"
  parsed = parser.from_file(pdf_path)
  txt = open('output.txt', 'w', encoding = 'utf-8')
  # output.txt에 pdf파일 내용을 write
  content_list.append(parsed['content'])

  txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# xlsx 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xlsx')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_xlsx_df = pd.DataFrame()


for i in filelist_xlsx:
    csv_xlsx_df = pd.read_excel(f"{i}")

full_text = []
for column in csv_df:
    for item in csv_df[column]:

        full_text.append(str(item))



csv_xlsx_df['title'] = np.nan
csv_xlsx_df['title'] = listToString(names)
csv_xlsx_df['content'] = listToString(full_text)
csv_xlsx_df['extension'] = 'csv'
csv_xlsx_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

In [143]:
df_compare_merged = pd.concat([hwp_df, docx_df, pptx_df,csv_df,pdf_df,csv_xlsx_df])
df_compare_merged = df_compare_merged.reset_index()
df_compare_merged = df_compare_merged.drop(['index'],axis=1)
for i in range(len(df_compare_merged['content'])):
  df_compare_merged['content'].loc[i] = remove_special_characters(df_compare_merged['content'].loc[i])

In [149]:
df_compare_merged

,title,content,extension
0,2022년 국정감사 업무보고.hwp,행정안전위원회 국정감사\r\n2022 10 4\r\n\r\n주요 업무 현황\r\n\...,hwp
1,2022 공무원보수 등의 업무지침(안) 전문...,제 1 장공무원 봉급업무 처리기준\r\n \r\n\r\n목 차\r\n\r\n ...,hwp
2,2023 공무원보수등의 업무지침 전문(최종...,제 1 장공무원 봉급업무 처리기준\r\n \r\n\r\n목 차\r\n\r\n ...,hwp
3,스트레스2.docx,스트레스는 삶의 다양한 도전이나 요구에 대한 신체의 일반적이고 자연스러운 반응입니다...,docx
4,발표용_v2.pptx,5 조 코딩메이트 이정섭 윤승현 이혁수 상품명 표준화 Contents 목...,pptx
5,5조 - 오더퀸 .xls,rakun rakun rakun rakun rakun rakun rakun raku...,csv
6,(금천2기)기업데이터를 활용한 AI 취업 ᄇ...,과정명\n\n과정번호 1495 확인란\n\n수강번호 27842 아이디 ysh2136...,pdf
7,"[23-062](보도자료) 외교부, 2030 부산세계박라...",외교부는 박진 장관 주재로 130월 2030 부산세계박람회 유치교섭 \n\n점검 특...,pdf
8,[23-063](보도자료) 한일 국장 협의(1.30) ᄀ...,서민정 외교부 아태국장은 130월 오후 후나코시 다케히로 \n 외무성 아시아대양주국...,pdf
9,[23-064](보도자료) OPCW 시리아 화학무기 조사ᄒ...,정부는 시리아 화학무기 사용 관련 127금 화학무기금지기구OPCW \n\n조사확인팀...,pdf
